## Want to arrange the cold hands estimate percentage vs player. If we order it by decreasing percentage, does anything obvious pop out (like is cold hands magnitude inversely correlated with player rank? -- could make sense because "better" players get in their heads less.)